ECS 174 Project

In [ ]:
#imports
import os
import numpy as np
from PIL import Image 
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix




: 

#1. Set dataset root and image size

In [ ]:
dataSetRoot = "DATASET"
# Root folder of the dataset 
# DATASET/
#   TRAIN/
#       O/
#       R/
#   TEST/
#       O/
#       R/


In [ ]:
sizeOfImage = (64, 64) 
##Preprocess a single image
def preprocess_image(path, size= sizeOfImage):
    img = Image.open(path).convert("RGB")   # ensure 3 channels (RGB)

    # resize to size (width, height)
    img = img.resize(size) # PIL resize

    # convert to float32 array in [0,1]
    img_np = np.array(img).astype(np.float64) / 255.0 # float64 to avoid runtime and convergence warning

    # flatten (64 * 64 * 3)
    img_flat = img_np.reshape(-1)
    return img_flat


##Load images for one label (O or R) in a split
def load_label(split_name, label_folder, label_value):
    X_list = []
    y_list = []

    folder = os.path.join(dataSetRoot, split_name, label_folder)

    if not os.path.isdir(folder):
        print("Folder does not exist:", folder)

    for fname in os.listdir(folder):
        path = os.path.join(folder, fname)
        img_flat = preprocess_image(path)
        X_list.append(img_flat)
        y_list.append(label_value)

    X = np.array(X_list, dtype=np.float64) # float64 to avoid runtime and convergence warning
    y = np.array(y_list, dtype=int)

    return X, y


In [ ]:
# Load all images of label  and combine 
def load_split(split_name):
    X_O, y_O = load_label(split_name, "O", 0)
    X_R, y_R = load_label(split_name, "R", 1)

    X = np.concatenate([X_O, X_R], axis=0)
    y = np.concatenate([y_O, y_R], axis=0)
    return X, y


In [ ]:
# load the data
X_train_full, y_train_full = load_split("TRAIN")
X_test, y_test = load_split("TEST")


#to check dataset size
# print("X_train_full shape:", X_train_full.shape)
# print("y_train_full counts:",
#       "O:", (y_train_full == 0).sum(),
#       "R:", (y_train_full == 1).sum())

# print("X_test shape:", X_test.shape)
# print("y_test counts:",
#       "O:", (y_test == 0).sum(),
#       "R:", (y_test == 1).sum())

#2 Split train, validation, test

https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html

In [ ]:
#Code
#Split into train, validation, test
X_train, X_val, y_train, y_val = train_test_split(
    X_train_full,
    y_train_full,
    test_size=0.2,
    stratify=y_train_full,
    random_state=0
)


In [ ]:
#Standarize Features

from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train) #mean/std based on each feature
X_val   = scaler.transform(X_val)
X_test  = scaler.transform(X_test)

#3. Train Logistic Regression

https://www.geeksforgeeks.org/machine-learning/understanding-logistic-regression/

In [ ]:
#Code
logRegModel = LogisticRegression(
    max_iter=2000,   #up the number of iterations to improve the convergence 
    random_state=0
)
logRegModel.fit(X_train, y_train)

#4. Evaluate

In [ ]:
#pred
ytest_pred = logRegModel.predict(X_test)

#Test accuracy
test_acc = accuracy_score(y_test, ytest_pred)
print("Test accuracy:", test_acc)

# metrics (for report) 
print("\nTest Metrics report (O = 0, R = 1):")
print(classification_report(y_test, ytest_pred, target_names=["O", "R"]))

# confusion matrix 
print("\nConfusion matrix (rows = true, cols = predicted):")
print(confusion_matrix(y_test, ytest_pred))

#5: Advance Project (PIPELINE)
- Take an image the model predicted as recyclable and convert to grayscale
- Apply blur, find edges (vertical segments)
- Draw a rectangle around it and show it alongside the prediction

In [ ]:
#Code

#used from hw1
def rgb_to_gray(image):
    return (0.2125 * image[:,:, 0].astype(np.float32) +
            0.7154 * image[:,:, 1].astype(np.float32) +
            0.0721 * image[:,:, 2].astype(np.float32))

def GaussianBlurImage(image, sigma):
    ## define Gaussian filter
    #got gaussian filter code from filtering.ipynb exactly till convolutio function called
    G_filter_size = 2 * int(4 * sigma + 0.5) + 1
    gaussian_filter = np.zeros((G_filter_size, G_filter_size), np.float32)
    for i in range(G_filter_size):
      for j in range(G_filter_size):
        x = i - G_filter_size // 2
        y = j - G_filter_size // 2
        gaussian_filter[i, j] = 1 / (2 * np.pi * sigma ** 2) * np.exp(-(x ** 2 + y ** 2)/(2 * sigma ** 2))

    ## and convolve
    img_blurred = convolution(image, gaussian_filter) # added filter to function

    return img_blurred

def SobelImage(image):
  #lecture4- Edges: Slide 8
  #formula also found on lec4(slide 8) and  (slide 28,29)
    gx= np.array([[-1,0,1],[-2,0,2],[-1,0,1]], dtype=np.float32)
    gy = np.array([[-1,-2,-1],[0,0,0],[1,2,1]], dtype=np.float32)
    #pass sobel filters into convolution
    magnitude = np.sqrt((convolution(image, gx)**2) + (convolution(image, gy)**2))
    orientation = np.arctan2(-(convolution(image, gy)), convolution(image, gx)) #arctan2 for angle over all quadrants, negative as per slides
    vertical_edge = convolution(image, gx)
    horizontal_edge = convolution(image, gy)

    return magnitude, orientation, vertical_edge, horizontal_edge

def convolution(image, filter):
    img_convolved = np.zeros_like(image, dtype=np.float32)
    ## if needed, pad image
     ## convolve
     #for boarders, 2D is greyscale and 3D is RBG
    padTwoD = ((filter.shape[0]//2, filter.shape[0]//2), (filter.shape[1]//2, filter.shape[1]//2)) #filtering.ipynb
    padThreeD = ((filter.shape[0]//2, filter.shape[0]//2), (filter.shape[1]//2, filter.shape[1]//2), (0,0))#filtering.ipynb
    imRow = image.shape[0] #from sample_code.py, # of rows
    imCol = image.shape[1]

    match image.ndim: #ndim = dimensions of array : discussion week 2 notebook
      case 3: #3D for color,
        return threedConvole (image, filter, padThreeD, imRow, imCol, img_convolved)

      case 2: #2D for no color
        return greyTwoDConvole (image, filter, padThreeD, imRow, imCol, img_convolved)
       
def threedConvole (image, filter, padThreeD, imRow, imCol, img_convolved):
  imagePad = ownPad(image,padThreeD, 'constant', 0).astype(np.float32)#filtering.ipynb of pad fucntion created
  for ch in range(image.shape[2]): #3 loops for each R, B, G
    for i in range(imRow):
      for j in range(imCol):
        frame = imagePad[i:i+filter.shape[0], j:j+filter.shape[1], ch] #slicing in discussion week 2 notebook
        img_convolved[i,j,ch] = np.sum(frame * filter)
  return img_convolved

def greyTwoDConvole (image, filter, padTwoD, imRow, imCol, img_convolved):
  imagePad = ownPad(image, padTwoD, 'constant', 0).astype(np.float32)
  for i in range(imRow): # two loops for gray scale
    for j in range(imCol):
      frame = imagePad[i:i+filter.shape[0], j:j+filter.shape[1]] ##slicing in discussion week 2 notebook
      img_convolved [i, j] = np.sum(frame * filter)
  return img_convolved

def ownPad (image, padW, mode, constval):
    #for corners
    leftTop = padW[0][0]
    rightTop = padW[0][1]
    leftBottom = padW[1][0]
    rightBottom = padW[1][1]
    imRow = image.shape[0]
    imCol = image.shape[1]

    # idea to create bigger matrix with zeros and paste the orignal image into center so expand + darker border
    match image.ndim:
      case 2:
        padIm = np.zeros((imRow + leftTop + rightTop, imCol + leftBottom + rightBottom), dtype=image.dtype) #zero matrix
        padIm[leftTop:leftTop+imRow, leftBottom:leftBottom+imCol] = image #orginal image
        return padIm

      case 3:
        padIm = np.zeros((imRow + leftTop + rightTop, imCol + leftBottom + rightBottom, image.shape[2]), dtype=image.dtype) #zero matrix
        padIm[leftTop:leftTop+imRow, leftBottom:leftBottom+imCol, :] = image #orginal image
        return padIm
        

In [ ]:
#recycling detection code

def recyclable_image(img_rgb, sigma_blur=2.0):

    gray = rgb_to_gray(img_rgb) #Convert to grayscale 
    guass_blurred = GaussianBlurImage(gray, sigma=sigma_blur)  # H x W
    magnitude, orientation, vertical_edge, horizontal_edge = SobelImage(guass_blurred) #edges
    vert_abs = np.abs(vertical_edge)

# to do: draw rectangle around


